In [1]:
import quandl
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [3]:
exchanges = ['KRAKEN','COINBASE','BITSTAMP','ITBIT']

exchanges_list = []
for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    df_exchange_temp = get_quandl_data(exchange_code)
    df_exchange_temp.reset_index(inplace=True)
    df_exchange_temp = df_exchange_temp.loc[(df_exchange_temp['Date'] >= datetime(2016, 1, 1)) & 
                                            (df_exchange_temp['Date'] < datetime(2018,1,1))]
    
    exchanges_list.append(df_exchange_temp)

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [4]:
df_total = pd.concat(exchanges_list)
df_total.sort_values('Date', inplace=True)
df_total.reset_index(drop=True, inplace=True)
df_total.head(16)

,Date,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,2016-01-01,434.17899,434.96999,430.74000,431.50531,14.348580,6.215768e+03,433.197419
1,2016-01-01,430.89000,436.00000,427.20000,433.82000,3788.111174,1.640578e+06,433.086003
2,2016-01-01,429.97000,435.41000,427.01000,433.65000,1083.936600,4.681026e+05,431.854189
3,2016-01-01,430.35000,437.15000,427.92000,435.66000,3863.277451,1.674183e+06,433.358101
4,2016-01-02,434.00000,434.99000,430.32476,434.98000,26.759482,1.158658e+04,432.989873
5,2016-01-02,434.87000,435.99000,430.42000,433.55000,2972.063449,1.287773e+06,433.292697
6,2016-01-02,435.67000,437.56000,432.41000,435.40000,3276.709621,1.426307e+06,435.286346
7,2016-01-02,434.29000,435.41000,431.55000,433.83000,315.777500,1.367986e+05,433.211856
8,2016-01-03,435.40000,435.75000,425.02000,431.91000,3904.335318,1.681589e+06,430.697845
9,2016-01-03,433.20000,434.09000,424.06000,431.04000,4571.097038,1.959153e+06,428.595713


In [15]:
# open, high, low, close, and weighted price are averaged by date from all exchanges 
df_final = df_total.groupby('Date').mean()
df_final.drop(['Volume (BTC)','Volume (Currency)'], axis=1, inplace=True)
df_final.columns = ['open','high','low','close','weighted_price']

# high_low_spread = high - low
df_final['high_low_spread'] = df_final['high'] - df_final['low']

# midquote_price = (high-low)/2
df_final['midquote_price'] = df_final['high_low_spread']/2

# effective_spread = 2 * (close - midquote_price)
df_final['effective_spread'] = 2 * (df_final['close'] - df_final['midquote_price'])

df_final = df_final.round(2)

# volume (btc and currency) are sum by date from all exchanges
df_final = pd.concat([df_final, df_total.groupby('Date').sum()[['Volume (BTC)','Volume (Currency)']]], axis=1, 
                   join_axes=[df_final.index])
df_final = df_final.round({'Volume (Currency)': 2})
df_final.rename(columns = {'Volume (BTC)':'volume_btc','Volume (Currency)':'volume_currency'},inplace=True)

df_final.head(10)

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,volume_btc,volume_currency
Date,,,,,,,,,,
2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,8749.673805,3789078.83
2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,6591.310053,2862465.49
2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,9678.402186,4157151.63
2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,13463.971066,5832898.79
2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,10381.184703,4492484.21
2016-01-06,432.20,432.44,427.21,429.90,430.16,5.23,2.62,854.56,12809.357391,5519290.27
2016-01-07,429.76,459.12,429.25,458.12,447.85,29.87,14.94,886.37,34986.550228,15701270.82
2016-01-08,457.84,463.33,446.58,453.88,454.98,16.75,8.37,891.00,21181.179401,9636610.85
2016-01-09,453.13,456.18,447.84,449.85,451.66,8.34,4.17,891.36,9862.746028,4446701.08


In [ ]:
df.to_csv(filename)